In [1]:
import os
import sys
import pandas as pd

import evo_helpers as hlp
import evo_constants as CTS

In [2]:
hlp.welcome()

Welcome to EvoMinig

Press enter to continune ...


In [ ]:
#from importlib import reload
#reload(evo_helpers)

opt = hlp.option_menu()

In [ ]:
class Evomining:
    """This class creates an evomining object.
    Its methods permit to access to the evomining
    funtionality.
    """

    def __init__(self, centralDB,
                evo_genomesDB,
                nat_prodsDB):
        self.central = centralDB
        self.genomesDB = evo_genomesDB
        self.nat_prods = nat_prodsDB

In [ ]:
## obtener las bases de datos del usuario

#from importlib import reload
#reload(evo_helpers)

## option 1 precharged example:
if opt == 1:
    hlp.make_all_evo_headers(CTS.EXMPL_DB["genomes"],
                                 CTS.EXMPL_DB["rast_ids"])
    hlp.join_evo_headers_files()
    hlp.remove_evo_headers_files()
    
    evo_genomes_path = CTS.EVO_GENOMES_DB
    
    evomining = EvoMining(cts.EXML_DB["central"],
                          cts.EXMPL_DB["genomes"],
                          cts.EXMPL_DB["nat_prods"])
    #evomining.run()
## option 2 user databases:
else: 
    user_dbs = hlp.get_dbs()
    hlp.make_all_evo_headers(user_dbs["genomes"], 
                             user_dbs["rast_ids"])
    hlp.join_evo_headers_files()
    hlp.remove_evo_headers_files()
    
    user_dbs["genomes"] = CTS.EVO_GENOMES_DB
    
    evomining = EvoMining(user_dbs["central"],
                         user_dbs["genomes"],
                         user_dbs["nat_prods"])
    #evomining.run()

#evomining.run()

## Obtener los archivos .faa con los encabezados en formato evomining:
### Los nuevos .faa individuales se guardan en data/aux_data/  (se borran una vez utilizados)
### La base de datos con encabezados de evomining se guarda en /data/data_bases/evo_genomes_db/

In [2]:
#from importlib import reload
#reload(evo_helpers)

genomes_db = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/example/genomes/los17"
Rast_ids = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/example/genomes/los17Rast.ids"
hlp.make_all_evo_headers(genomes_db, Rast_ids)
hlp.join_evo_headers_files()
hlp.remove_evo_headers_files()

## Aplicar makeblastdb a las bases de datos que necesitaremos:

In [3]:
hlp.makeblast_db(CTS.EVO_GENOMES_DB)

In [4]:
hlp.makeblast_db(CTS.EXMPL_DB["central"])


In [5]:
hlp.makeblast_db(CTS.EXMPL_DB["nat_prods"])

## Obtener los blastp entre la base de datos central vs la base de datos de genomas:

In [6]:
blastdb_path = CTS.BLASTDBs_PATH + 'evo_genomes_db_blastdb/'
            
hlp.apply_blastp(CTS.EXMPL_DB["central"], blastdb_path)

In [7]:
CTS.EVO_GENOMES_DB

'data/data_bases/evo_genomes_db/'

## Obtener los blastp entre la base de datos de genomas vs la base de datos central:

In [8]:
blastdb_path = CTS.BLASTDBs_PATH + 'central_blastdb/'
            
hlp.apply_blastp(CTS.EVO_GENOMES_DB, blastdb_path)

## Obtener los BBH:

### Obtener los mejores elementos únicos del blast de metabolismo central vs genomas: 
### Estos se guardarán en un diccionario de la forma:
### { enzima_i : mejor_copia_en_el_organismo_k }
### donde i recorre todas las enzimas en el metabolismo central, y k recorre todos los genomas.
### - Considerar solo con bitscore >= 100 e E_value < 0.001 

In [3]:
df1 = pd.read_csv('/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/central_to_genomes/central_to_genomes.blast', sep = "\t", names = CTS.BLAST_COLS)

In [38]:
central_enzimes = df1['query'].unique() ##Lista con las enzimas del metabolismo central
genomes = hlp.dict_ids_to_names_genomes(CTS.EXMPL_DB["rast_ids"]) ## Diccio con los genomas de la base de genomas

In [41]:
df1.head()

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.146852.1090|6666666.146852|NC_00293...,79.623,530,108,0,1,530,1,530,0.0,847.0
1,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|749927.13.1727|749927.13|NC_014318.1|D-3-ph...,59.586,532,213,2,1,530,1,532,0.0,620.0
2,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|1156913.7.6305|1156913.7|NC_021252.1|D-3-ph...,59.203,527,214,1,1,526,1,527,0.0,613.0
3,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.104540.4265|6666666.104540|AOHO01|D...,59.203,527,214,1,1,526,1,527,0.0,612.0
4,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|446462.15.6083|446462.15|NC_021252.1|D-3-ph...,60.338,532,209,2,1,530,1,532,0.0,595.0


In [6]:
#definimos el diccionario que será la salida:
unique_in_first_blast = dict()
#inicializamos el diccionario tomando por claves todas las enzimas
# y por el momento asignando por valores listas vacías:
for enzime in central_enzimes:
    unique_in_first_blast[enzime] = list()
    
#definimos una lista auxiliar que nos permitirá quedarnos con la mejor
# copia de la enzima presente en cada organismo 

aux_list = list()
for enzime in central_enzimes:
    for genome_id in genomes.keys():
        for index in range(len(df1)):
            if (genome_id in df1["subject"][index]) & (aux_list == []):
                aux_list=list(df1.iloc[index])
            elif ((genome_id in df1["subject"][index]) & (df1["bitscore"][index] > aux_list[11])):
                aux_list = list(df1.iloc[index])
            else:
                continue
    unique_in_first_blast[enzime].append(aux_list)
    

IndexError: list index out of range

In [7]:
central_enzimes

array(['3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu',
       '3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_2|Mtub',
       '3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_3|Mtub',
       '3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe',
       '3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_1|none',
       '3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_2|none',
       '3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none'],
      dtype=object)

In [8]:
genomes.keys()

dict_keys(['6666666.385189', '1156913.7', '169133.3', '290340.24', '367928.21', '442563.11', '446462.15', '452863.24', '649831.6', '6666666.104540', '6666666.111158', '6666666.111572', '6666666.112811', '6666666.146813', '6666666.146852', '6666666.146872', '749927.13', '930171.11'])

In [14]:
#definimos el diccionario que será la salida:
unique_in_first_blast = dict()
#inicializamos el diccionario tomando por claves todas las enzimas
# y por el momento asignando por valores listas vacías:
for enzime in central_enzimes:
    unique_in_first_blast[enzime] = []
    
#definimos una lista auxiliar que nos permitirá quedarnos con la mejor
# copia de la enzima presente en cada organismo 

#aux_list = list()
for enzime in central_enzimes:
    for genome_id in genomes.keys():
        aux_list = []
        if genome_id != "6666666.385189":
            for index in range(len(df1)):
                if (aux_list == []):
                    if ((enzime == df1["query"][index])&(genome_id in df1["subject"][index])):
                        aux_list = list(df1.iloc[index])
                    else:
                        continue
                else:
                    if ((enzime == df1["query"][index]) & (genome_id in df1["subject"][index]) & (df1["bitscore"][index] > aux_list[11])):
                        aux_list = list(df1.iloc[index])
            if aux_list != []:  
                unique_in_first_blast[enzime].append(aux_list)
                


In [15]:
unique_in_first_blast["3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none"]

[['3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
  'gi|1156913.7.7342|1156913.7|NC_021252.1|Phosphoserine_aminotransferase_(EC_2.6.1.52)|Amycolatopsis_orientalis_HCCB10007_NC_021252.1',
  67.769,
  363,
  116,
  1,
  1,
  363,
  13,
  374,
  3.0300000000000004e-179,
  503.0],
 ['3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
  'gi|169133.3.2262|169133.3|LAB|Phosphoserine_aminotransferase_(EC_2.6.1.52)|Citricoccus_sp_CH26_LAB',
  68.32,
  363,
  115,
  0,
  1,
  363,
  12,
  374,
  0.0,
  524.0],
 ['3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
  'gi|290340.24.1033|290340.24|NC_008711.1|Phosphoserine_aminotransferase_(EC_2.6.1.52)|Arthrobacter_aurescens_TC1_NC_008711.1',
  67.493,
  363,
  118,
  0,
  1,
  363,
  12,
  374,
  0.0,
  519.0],
 ['3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none',
  'gi|367928.21.407|367928.21|NC_008618.1|Phosphoserine_aminotransferase_(EC_2.6.1.52)|Bifidobacterium_adolescentis_ATCC_15703_NC_008618.1',
  57.297,
  3

In [13]:
total = 0
for enz in central_enzimes:
        total = total + len(unique_in_first_blast[enz])
total

119

### NOTA: En la base de datos de genomas está de más el genoma con id = 6666666.385189

Unique de Genomes vs EF:

In [11]:
df2 = pd.read_csv('/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/genomes_to_central/genomes_to_central.blast', sep = "\t", names = CTS.BLAST_COLS )

In [12]:
#definimos el diccionario que será la salida:
unique_in_second_blast = dict()
#inicializamos el diccionario tomando por claves todas las enzimas
# y por el momento asignando por valores listas vacías:
for genome_id in genomes.keys():
    if genome_id != "6666666.385189":
        unique_in_second_blast[genome_id] = []
    else:
        continue

#definimos una lista auxiliar que nos permitirá quedarnos con la mejor
# copia de la enzima presente en cada organismo 


for genome_id in genomes.keys():
    if genome_id != "6666666.385189":
        for enzime in central_enzimes:
            aux_list = []
            for index in range(len(df2)):
                if (aux_list == []):
                    if ((genome_id in df2["query"][index])&(enzime in df2["subject"][index])):
                        aux_list = list(df2.iloc[index])
                    else:
                        continue
                else:
                    if ((genome_id in df2["query"][index]) & (enzime in df2["subject"][index]) & (df2["bitscore"][index] > aux_list[11])):
                    
                        aux_list = list(df2.iloc[index])
            if aux_list != []:  
                unique_in_second_blast[genome_id].append(aux_list)


### Prueba de bio.genbank package:

In [19]:
from taxonomy_ranks import TaxonomyRanks

In [30]:
taxa_name = 'Actinomadura hibisca'

rank_taxon = TaxonomyRanks(taxa_name)

In [31]:
type(rank_taxon)

taxonomy_ranks.get_taxonomy_rank_with_ete3_with_super_and_sub.TaxonomyRanks

In [32]:
rank_taxon.get_lineage_taxids_and_taxanames()


In [33]:
ranks = ('user_taxa', 'taxa_searched', 'superkingdom', 'kingdom', 'superphylum', 'phylum', 'subphylum', 'superclass', 'class', 'subclass', 'superorder', 'order', 'suborder', 'superfamily', 'family', 'subfamily', 'genus', 'subgenus', 'species')

In [34]:
for potential_taxid in rank_taxon.lineages:
     for rank in ranks:
         if rank in ('user_taxa', 'taxa_searched'):
             taxon = rank_taxon.lineages[potential_taxid][rank]
             print(potential_taxid, rank, taxon, sep='\t')
         else:
             taxon, taxid_of_taxon = rank_taxon.lineages[potential_taxid][rank]
             print(potential_taxid, rank, taxon, taxid_of_taxon, sep='\t')


68565	user_taxa	Actinomadura hibisca
68565	taxa_searched	Actinomadura hibisca
68565	superkingdom	Bacteria	2
68565	kingdom	NA	NA
68565	superphylum	NA	NA
68565	phylum	Actinomycetota	201174
68565	subphylum	NA	NA
68565	superclass	NA	NA
68565	class	Actinomycetes	1760
68565	subclass	NA	NA
68565	superorder	NA	NA
68565	order	Streptosporangiales	85012
68565	suborder	NA	NA
68565	superfamily	NA	NA
68565	family	Thermomonosporaceae	2012
68565	subfamily	NA	NA
68565	genus	Actinomadura	1988
68565	subgenus	NA	NA
68565	species	Actinomadura hibisca	68565
